#### Website name : Acko drive

In [29]:
import requests, numpy as np, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import time

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [20]:
headers = {"Accept-Language": "en-US,en;q=0.9", 'User-Agent':'Mozilla/5.0'}

In [67]:
driver = webdriver.Chrome(service =ChromeService(ChromeDriverManager().install()))

In [68]:
driver.get("https://ackodrive.com/cars/")

In [69]:
names =[]
seats = []
variants = []
fuel_type = []
variant_transmission = []
city = []
price = []
model_type = []

driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[1]/div[2]/div[1]/span').click()  # clicks on city selection menu
time.sleep(5)

for cty in range(1,12):  # loop to select city from 1 to 11
   
    path = f'//*[@id="__next"]/div/div/div[1]/div[3]/div/div/div/div[2]/div[2]/a[{cty}]/div/span[1]/img'  # path for particular city
        
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, path))
    )
    element.click()  # clicks on particular city
    time.sleep(10)
    
    try:    
        for j in range(2,6):   # loop to fetch info from page 2 to 5 for each city 
             
            pt = f'//*[@id="cars-list-container"]/div[2]/div[2]/div/div[3]/ul/li[{j+1}]/a'
            driver.find_element(By.XPATH, pt).click()  # clicks on particular page
            
            time.sleep(5)
            
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            
            container = soup.find_all('div', class_="styles__Wrapper-sc-cbcba852-0 jtAUjc")
            
            for i in container:  # loop to select container in a page
                
                # fetches the name of cars
                
                n = i.find('h2', class_="styles__ModelTitle-sc-57d31ed8-4 LQrbf")
                names.append(n.text)
            
                # fetches seater and number of variants
                
                s = i.find('div', class_="styles__ModelInfo-sc-57d31ed8-8 bMwutE")
            
                if s is not None:
                    reg = re.findall(r'(\d)\sSeater', s.text)
                    if len(reg)>0:
                        seats.append(reg[0])
        
                    else:
                        seats.append(np.nan)
        
                    v = re.findall(r'(\d+)\svariants', s.text)
                    if len(v)>0:
                        variants.append(v[0])
        
                    else:
                        variants.append(np.nan)           
                
                else:
                    seats.append(np.nan)
                    variants.append(np.nan)
        
        
                # fetches model type
                rx = re.findall(r'^(\w+)\s', s.text)
                if len(rx)>0:
                    model_type.append(rx[0])
                else:
                    model_type.append(np.nan)
            
                # fetches fuel type and #transmission type
                x  = i.find('div', class_ = 'styles__VariantInfo-sc-57d31ed8-12 fuepeG')
                
                r = re.match(r'^(Petrol|Diesel|Electric|CNG)',x.text)
                
                if r is not None:
                    fuel_type.append(r.group(1).strip())  
                else:
                    fuel_type.append(np.nan)
            
                t = re.search(r'(Manual|Automatic)',x.text)
                
                if t is not None:
                    variant_transmission.append(t.group(1).strip())
        
                else:
                    variant_transmission.append(np.nan)
                
                # fetches the city 
                ct  = i.find('div', class_ = 'styles__CityName-sc-57d31ed8-17 lhXIXl')
                
                rg = re.findall(r'\w+\s(\w+)',ct.text)
                
                if len(rg)>0:
                    city.append(rg[0])
                else:
                    city.append(np.nan)
            
                # fetches the price
                pr  = i.find('div', class_ = 'styles__Price-sc-57d31ed8-18 btbaoV')
                
                rgx = re.findall(r'\d+\.\d+',pr.text)
                if len(rgx)>0:
                    price.append(rgx[0])
                else:
                    price.append(np.nan)

    except Exception as e:
        print(f"Could not click on element for city {cty} at page{j}: {e}")
        continue



    driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[1]/div[2]/div[1]/span').click()  # clicks on city selection menu 
    time.sleep(4)
    

driver.quit()
            

In [70]:
print(len(price))
print(len(city))
print(len(variant_transmission))
print(len(fuel_type))
print(len(model_type))
print(len(variants))
print(len(names))
print(len(seats))

626
626
626
626
626
626
626
626


In [71]:
# creating dataframe

dictn = {'carName':names,'carPrice':price, 'seater':seats, 'city':city, 'variantTransmission':variant_transmission,'fuelType':fuel_type,'modelType':model_type,'numberOfvariants':variants }
df = pd.DataFrame(dictn)

In [72]:
print(df)

                         carName carPrice seater       city  \
0                  Tata Tiago EV     8.91      5  Ahmedabad   
1                      Kia Sonet     9.12      5  Ahmedabad   
2         Maruti Suzuki S-Presso     5.63      5  Ahmedabad   
3                     Tata Tiago     6.08      5  Ahmedabad   
4                     Citroen C3     7.06      5  Ahmedabad   
..                           ...      ...    ...        ...   
621              Mahindra Bolero    11.47      7       Pune   
622         Hyundai Creta N Line    21.78      5       Pune   
623  Toyota Urban Cruiser Taisor     9.23      5       Pune   
624                Mahindra Thar    14.96      4       Pune   
625               Citroen Basalt     9.46      5       Pune   

    variantTransmission  fuelType   modelType numberOfvariants  
0             Automatic  Electric  Hatchback5                7  
1                Manual    Petrol        SUV5               24  
2                Manual    Petrol        SUV5   

In [74]:
# storing dataframe in csv file
df.to_csv('cars.csv')

In [73]:
df['city'].unique()

array(['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Faridabad',
       'Ghaziabad', 'Gurgaon', 'Hyderabad', 'Mumbai', 'Noida', 'Pune'],
      dtype=object)

In [18]:
z = []
a = soup.find_all('div', class_="styles__VariantPrice-sc-57d31ed8-15 dWaoEb")

for k in a:
    z.append(k.text)


In [19]:
z

['On-road pricein Bangalore₹21.45 LExpress deliverywithin 11 days ',
 "On-road pricein Bangalore₹43.18 LGet it by26 Aug'26",
 'On-road pricein Bangalore₹16.15 LExpress deliverywithin 11 days ',
 "On-road pricein Bangalore₹24.22 LGet it by26 Aug'26",
 "On-road pricein Bangalore₹41.95 LGet it by24 Nov'24",
 "On-road pricein Bangalore₹54.64 LGet it by24 Nov'24",
 "On-road pricein Bangalore₹11.77 LGet it by04 Nov'24",
 "On-road pricein Bangalore₹18.94 LGet it by24 Nov'24",
 "On-road pricein Bangalore₹9.88 LGet it by26 Aug'26"]

In [39]:
pip install webdriver-manager